In [3]:
from binance_bot_class  import BinanceBot
from pprint import pprint
from binance.async_client import AsyncClient
from dotenv import load_dotenv
import numpy
import pandas as pd
import pandas_ta as ta
import os

load_dotenv()
API_KEY = os.getenv("BINANCE_API_KEY")
API_SECRET = os.getenv("BINANCE_API_SECRET")

bot = BinanceBot()
await bot.initialize_client()

client = await AsyncClient.create(API_KEY, API_SECRET)


/Users/gimhyeongjun/Desktop/programs/DevProjects/binance-future2/.venv/lib/python3.12/site-packages/pandas_ta/utils/_core.py:14: SyntaxWarning: invalid escape sequence '\g'
  return re_.sub("([a-z])([A-Z])","\g<1> \g<2>", x).title()


In [4]:
# 티커정보 가져오기
try:
    client = await AsyncClient.create(API_KEY, API_SECRET)
    balance = await client.get_asset_balance(asset='USDT')
    pprint(balance)
finally:
    await client.close_connection()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10b81eb70>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x11fabe1b0>, 12223.27720525)])']
connector: <aiohttp.connector.TCPConnector object at 0x10b81eab0>


BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [10]:
try:
    # 모든 포지션 정보 가져오기
    positions = await client.futures_position_information()
    
    # 보유 포지션만 필터링
    active_positions = [p for p in positions if float(p['positionAmt']) != 0]
    
    # 출력
    for pos in active_positions:
        print(f"Symbol: {pos['symbol']}")
        print(f"Position Amount: {pos['positionAmt']}")  # 포지션 크기
        print(f"Entry Price: {pos['entryPrice']}")       # 진입 가격
        print(f"Unrealized Profit: {pos['unRealizedProfit']}")  # 미실현 손익
        print(f"Margin Type: {pos['marginType']}")       # 마진 타입 (Isolated/ Cross)
        print(f"Leverage: {pos['leverage']}")           # 레버리지
        print("-" * 30)

except Exception as e:
    print(f"Error fetching positions: {e}")
finally:
    await client.close_connection()

In [ ]:
positions = await client.futures_position_information()
print(positions)

In [ ]:
# 티커정보 가져오기
try:
    info = await bot.ticker_info(symbol="CRVUSDT")
    pprint(info)
finally:
    await bot.close()

In [ ]:
# 티커정보 가져오기
try:
    account = await bot.update_account()
    pprint(account)
finally:
    await bot.close()

In [ ]:
try:
    data = await bot.futures_historical_klines(
        symbol="BTCUSDT",
        interval='1d',
        start_str='2021-11-24',
        end_str="2021-11-26"
    )
    print(data)
finally:
    await bot.close()

In [6]:

new_tickers_info = []
symbol_list = ["CRVUSDT","CHRUSDT","RUNEUSDT","ARUSDT","FARTCOINUSDT"]
for symbol in symbol_list:
    # 200개 봉 데이터 가져오기
    klines = await client.futures_klines(symbol=symbol, interval="1d", limit=200)
    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume", "close_time",
        "quote_asset_volume", "trades", "taker_base_volume", "taker_quote_volume", "ignore"
    ])
    df["close"] = df["close"].astype(float)
    df["high"] = df["high"].astype(float)
    df["low"] = df["low"].astype(float)

    # 현재가 가져오기
    current_price = df["close"].iloc[-1]

    # RSI 계산 (14개 기간 기준)
    df["rsi"] = ta.rsi(df["close"], length=14)
    latest_rsi = df["rsi"].iloc[-1]

    # 최근 80개 봉 최고가/최저가 계산
    highest_80 = df["high"].iloc[-80:].max()
    lowest_80 = df["low"].iloc[-80:].min()

    # 현재가가 최고/최저가 대비 위치 (%)
    position_percentage = (current_price - lowest_80) / (highest_80 - lowest_80) * 100

    # 데이터 저장
    new_tickers_info.append({
        "symbol": symbol,
        "current_price": current_price,
        "rsi": latest_rsi,
        "highest_80": highest_80,
        "lowest_80": lowest_80,
        "position_percentage": position_percentage
    })

# 업데이트된 티커 정보 반영
pprint(new_tickers_info)

[{'current_price': 0.671,
  'highest_80': 1.338,
  'lowest_80': 0.276,
  'position_percentage': 37.1939736346516,
  'rsi': 38.363830678573045,
  'symbol': 'CRVUSDT'},
 {'current_price': 0.1446,
  'highest_80': 0.3765,
  'lowest_80': 0.1407,
  'position_percentage': 1.65394402035624,
  'rsi': 30.942891197178938,
  'symbol': 'CHRUSDT'},
 {'current_price': 1.316,
  'highest_80': 7.586,
  'lowest_80': 0.95,
  'position_percentage': 5.515370705244124,
  'rsi': 28.51170269985495,
  'symbol': 'RUNEUSDT'},
 {'current_price': 13.881,
  'highest_80': 28.988,
  'lowest_80': 12.28,
  'position_percentage': 9.582236054584637,
  'rsi': 43.06519664733937,
  'symbol': 'ARUSDT'},
 {'current_price': 0.632,
  'highest_80': 2.7489,
  'lowest_80': 0.5762,
  'position_percentage': 2.568233074055321,
  'rsi': 38.221719558627385,
  'symbol': 'FARTCOINUSDT'}]
